##1. Install and Import Dependencies

In [ ]:
!pip install transformers requests beautifulsoup4 pandas numpy

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

##2. Instantiate Model

In [ ]:
!pip uninstall torchvision -y
!pip install torchvision==0.21.0+cu118 --index-url https://download.pytorch.org/whl/cu118

Found existing installation: torchvision 0.21.0+cu118
Uninstalling torchvision-0.21.0+cu118:
  Successfully uninstalled torchvision-0.21.0+cu118
Looking in indexes: https://download.pytorch.org/whl/cu118
  Using cached https://download.pytorch.org/whl/cu118/torchvision-0.21.0%2Bcu118-cp311-cp311-linux_x86_64.whl.metadata (6.1 kB)
Using cached https://download.pytorch.org/whl/cu118/torchvision-0.21.0%2Bcu118-cp311-cp311-linux_x86_64.whl (6.5 MB)


In [ ]:
!pip uninstall transformers -y
!pip install transformers

Found existing installation: transformers 4.52.4
Uninstalling transformers-4.52.4:
  Successfully uninstalled transformers-4.52.4
  Using cached transformers-4.52.4-py3-none-any.whl.metadata (38 kB)
Using cached transformers-4.52.4-py3-none-any.whl (10.5 MB)


In [ ]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
#

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/669M [00:00<?, ?B/s]

##3. Encode and Calculate Sentiment

In [ ]:
tokens = tokenizer.encode('It was good but couldve been better. GREAT', return_tensors='pt')

In [ ]:
result = model(tokens)

In [ ]:
int(torch.argmax(result.logits))+1

4

##4.Collect Reviews

In [ ]:
# Fetch the page
r = requests.get('https://www.yelp.com/biz/social-brew-cafe-pyrmont')

# Parse the HTML
soup = BeautifulSoup(r.text, 'html.parser')

# Define regex to match review comments
regex = re.compile('.*comment.*')

# Find all relevant <p> tags with class matching the regex
results = soup.find_all('p', {'class': regex})

# Extract the text from each review
reviews = [result.text.strip() for result in results if result.text.strip()]

# Print the reviews
for review in reviews:
    print(review)

Very cute coffee shop and restaurant. They have a lovely outdoor seating area and several tables inside.  It was fairly busy on a Tuesday morning but we were to grab the last open table. The server was so enjoyable, she chatted and joked with us and provided fast service with our ordering, drinks and meals. The food was very good. We ordered a wide variety and every meal was good to delicious. The sweet potato fries on the Chicken Burger plate were absolutely delicious, some of the best I've ever had. I definitely enjoyed this cafe, the outdoor seating, the service and the food!!
Amazing ambience. Coffee was the best we've had in Sydney. The service and food also great. As must stop for breakfast.
Six of us met here for breakfast before our walk to Manly. We were enjoying visiting with each other so much that I apologize for not taking any photos. We all enjoyed our food, as well as our coffee and tea drinks.We were greeted immediately by a friendly server asking if we would like to si

##5. Load Reviews into Dataframe and Score

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.DataFrame(np.array(reviews), columns=['review'])

In [ ]:
df.head()

,review
0,Very cute coffee shop and restaurant. They hav...
1,Amazing ambience. Coffee was the best we've ha...
2,Six of us met here for breakfast before our wa...
3,Ricotta hot cakes! These were so yummy. I ate ...
4,We came for brunch and they ran out of seven s...


In [ ]:
df.tail()

,review
5,We came for brunch twice in our week-long visi...
6,Great food amazing coffee and tea. Short walk ...
7,Great place with delicious food and friendly s...
8,The food was delicious. The ricotta pancakes w...
9,"Great service, lovely location, and really ama..."


In [ ]:
df['review'].iloc[0]

"Very cute coffee shop and restaurant. They have a lovely outdoor seating area and several tables inside.  It was fairly busy on a Tuesday morning but we were to grab the last open table. The server was so enjoyable, she chatted and joked with us and provided fast service with our ordering, drinks and meals. The food was very good. We ordered a wide variety and every meal was good to delicious. The sweet potato fries on the Chicken Burger plate were absolutely delicious, some of the best I've ever had. I definitely enjoyed this cafe, the outdoor seating, the service and the food!!"

In [ ]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits)) + 1

In [ ]:
sentiment_score(df['review'].iloc[1])

5

In [ ]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))


In [ ]:
df

,review,sentiment
0,Very cute coffee shop and restaurant. They hav...,4
1,Amazing ambience. Coffee was the best we've ha...,5
2,Six of us met here for breakfast before our wa...,4
3,Ricotta hot cakes! These were so yummy. I ate ...,5
4,We came for brunch and they ran out of seven s...,2
5,We came for brunch twice in our week-long visi...,4
6,Great food amazing coffee and tea. Short walk ...,5
7,Great place with delicious food and friendly s...,5
8,The food was delicious. The ricotta pancakes w...,4
9,"Great service, lovely location, and really ama...",5


In [ ]:
df['review'].iloc[3]

"Ricotta hot cakes! These were so yummy. I ate them pretty fast and didn't share with anyone because they were that good ;). I ordered a green smoothie to balance it all out. Smoothie was a nice way to end my brekkie at this restaurant. Others with me ordered the salmon Benedict and the smoked salmon flatbread. They were all delicious and all plates were empty. Cheers!"